In [14]:
%load_ext autoreload

%autoreload 2

In [1]:
# gradient based (Langevin or Hamiltonian) HINTS

import torch
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.nn import functional as F

from HINTS_fn import *
from HINTS_torch import *

In [2]:
# first test PyTorch
train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = dsets.MNIST(root='./data', train=False, transform=transforms.ToTensor())
batch_size = 100
#train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw



In [3]:
# Example PyTorch model
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = self.linear(x) # pytorch Crossentropy loss takes scores, not logits as inputs
        #outputs = F.softmax(self.linear(x), dim = -1) # why better without F.softmax
        return outputs 

In [4]:
# was not worth caching this
def get_minibatch(dataset, n, i, input_dim = 784):
    lst1, lst2 = zip(*[dataset[ii] for ii in range(i * n, (i+1) * n)])
    xs = torch.stack(lst1).view(-1, input_dim)
    ys = torch.tensor(lst2)
    return(xs, ys)

In [5]:
%%time
input_dim = 784
output_dim = 10

# Note we will use a Bayesian model average of (some distribution over) history of sampled class probs in the sampling case
criterion = torch.nn.CrossEntropyLoss(reduction = 'sum') # computes softmax and then the cross entropy
model = LogisticRegression(input_dim, output_dim)
lr_rate = 0.001
optimizer = torch.optim.SGD(model.parameters(), lr=lr_rate) # SToch Gdt Descent
N = train_dataset.__len__()//batch_size # TO DO: get N from dataset
print(N)
epochs = 4 #n_iters / (len(train_dataset) / batch_size)
i = 0
for epoch in range(int(epochs)):
    epoch_losses = []
    for it in range(N):
        optimizer.zero_grad()
        images, labels = get_minibatch(train_dataset, batch_size, it)    
        outputs = model(images)
        loss = criterion(outputs, labels) # NEXT STEP: replace with an evaluate call!!!
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.detach().item())
        i += 1
    # end of epoch: calculate Accuracy
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.view(-1, 28*28))
        outputs = model(images)
        # TO DO convert these outputs to probs, to get more accurate Accuracy metric
        # and to support historical averaging (e.g. q(n) = p(n) * 2/(n+1) + q(n-1) * (n-1)/(n+1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # for gpu, bring the predicted and labels back to cpu fro python operations to work
        correct+= (predicted == labels).sum()
    accuracy = 100 * float(correct)/total
    print("Iteration: {}. Loss: {:1f}. Accuracy: {}.".format(i, torch.tensor(epoch_losses).mean(), accuracy))

600
Iteration: 600. Loss: 53.563038. Accuracy: 90.16.
Iteration: 1200. Loss: 35.924446. Accuracy: 90.94.
Iteration: 1800. Loss: 33.135147. Accuracy: 91.37.
Iteration: 2400. Loss: 31.675495. Accuracy: 91.61.
Wall time: 47.6 s


In [9]:
# Create the user function for HINTS


device = 'cuda' if torch.cuda.is_available() else 'cpu' # GPU slightly reduces minibatch wall clock time (100)

class TorchMNIST(UserFn):
        def __init__(self, additive = True):
            self.batch_size = 100 # 60000 dataset size (so will not see many GPU benefits)
            self.input_dim = 784
            self.output_dim = 10
            self.train_dataset = dsets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
            self.N = self.train_dataset.__len__()//self.batch_size # num scenarios
            self.lr = 0.001 # TO DO pass this parameter
            self.additive = additive # used by HINTS
            print(self.N)
            self.criterion = torch.nn.CrossEntropyLoss(reduction = 'sum')
            #self.calls = [] # for cache stats
            super().__init__(None)
        #
        def sample_initial_state(self):
            model = LogisticRegression(self.input_dim, self.output_dim).to(device)
            return(model)
        #    
        @lru_cache(maxsize = 1000000)
        def evaluate(self, state, term_index, gradient = False):
            #self.calls.append((hash(self), hash(state), hash(term_index), hash(gradient)))
            self.counter += 1
            if gradient:
                f = -self.minibatch_loss(state, term_index)
                f.backward()
            else:
                with torch.no_grad():
                    f = -self.minibatch_loss(state, term_index)
            return(f)
        #
        # this can be called with or without torch.no_grad
        def minibatch_loss(self, model, term_index):
            images, labels = get_minibatch(self.train_dataset, self.batch_size, term_index)  
            outputs = model(images.to(device))
            loss = self.criterion(outputs.cpu(), labels) # NEXT STEP: replace with an evaluate call!!!
            return(loss)



In [10]:
# test it   
MH = TorchMNIST()
state0 = MH.sample_initial_state()
state1 = state0
%time v = MH.evaluate(state0, 6, True)
%time v = MH.evaluate(state1, 6, True) # cached: should be faster
print(v)
for f in state0.parameters():
    print(f.shape, f.grad.shape)

600
Wall time: 27 ms
Wall time: 0 ns
tensor(-231.8298, grad_fn=<NegBackward0>)
torch.Size([10, 784]) torch.Size([10, 784])
torch.Size([10]) torch.Size([10])


In [11]:
%%time
# MINIBATCH SGD EXAMPLE (no HINTS yet)
epochs = 4
lr_rate = 0.001
model = MH.sample_initial_state()
optimizer = torch.optim.SGD(model.parameters(), lr = lr_rate)
i = 0
for epoch in range(int(epochs)):
    epoch_losses = []
    for it in range(MH.N):
        optimizer.zero_grad()
        loss = MH.minibatch_loss(model, it).cpu()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.detach().item())
        i += 1
    # end of epoch: calculate Accuracy
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = Variable(images.view(-1, 28*28))
        outputs = model(images.to(device)).cpu()
        # TO DO convert these outputs to probs, to get more accurate Accuracy metric
        # and to support historical averaging (e.g. q(n) = p(n) * 2/(n+1) + q(n-1) * (n-1)/(n+1))
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        # for gpu, bring the predicted and labels back to cpu fro python operations to work
        correct+= (predicted == labels).sum()
    accuracy = 100 * float(correct)/total
    print("Iteration: {}. Loss: {:1f}. Accuracy: {}.".format(i, torch.tensor(epoch_losses).mean(), accuracy))



Iteration: 600. Loss: 53.401009. Accuracy: 90.24.
Iteration: 1200. Loss: 35.948322. Accuracy: 90.98.
Iteration: 1800. Loss: 33.153053. Accuracy: 91.41.
Iteration: 2400. Loss: 31.689238. Accuracy: 91.59.
Wall time: 47.8 s


In [20]:
import argparse
parser = argparse.ArgumentParser()
args = parser.parse_known_args()[0] # defaults


if True: # set to True for HINTS
    args.levels = 6
    log_branch_factor = 1
    N_0 = 1
    args.design = np.array([1,2,3,2,5,2,5])
    NUM_SCENARIOS = args.design.prod()
    aa = False
    sd = 0.01
elif True: # test Langevin MCMC first (or set sd to small value for SGD)
    args.levels = 1 
    NUM_SCENARIOS = 600 # Langevin / SGD
    args.design = np.array([1,NUM_SCENARIOS]) # ensure whole dataset is covered
    aa = True # use always_accept flag to get minibatch Langevin (so we actually only have  a 1 level architecture) - TOP LEVEL ALWAYS ACCEPT
    sd = 0.001
else: # FULL DATASET (not minibatch) Langevin/gradient descent
    args.levels = 0 
    NUM_SCENARIOS = 600 # 600 for naive mcmc, 1 for SGD
    args.design = np.array([NUM_SCENARIOS]) # ensure whole dataset is covered
    aa = True # always accept only applies to TOP LEVEL
    sd = 0.001
    
print(NUM_SCENARIOS)

# design now has levels + 1 entries: so we can give a minibatch size in design[0]
# additive log probability is more natural from a Bayesian perspective but both are valid

args.additive = True # effectively selects a different temperature structure when False (= average or expectation)

args.T = 1.0 #top level
args.dT = 0.0 if args.additive else 0.5 # temperature increment by level (mainly for optimisation or averaging structure)
args.epsilon = 0.02 # for HMC
print(args.__dict__)


g = TorchMNIST(args.additive)
hmc = HINTS_HMC(args, g, noise_sd = sd) # noise sd crucial for acceptance rate (check maths for sd not equal to 1
state  = g.sample_initial_state()
print(state)



600
{'levels': 6, 'design': array([1, 2, 3, 2, 5, 2, 5]), 'additive': True, 'T': 1.0, 'dT': 0.0, 'epsilon': 0.02}
600
RESET
6
[  1   2   6  12  60 120 600]
600
[1. 1. 1. 1. 1. 1. 1.]
LogisticRegression(
  (linear): Linear(in_features=784, out_features=10, bias=True)
)


In [21]:
t = 0
time0 = time.time()
print(time0)
now = time0
then = now
while (now < (time0 + 1200.0)):
    hmc.shuffle()
    print(t)
    g.evaluate.cache_clear() # risk of revisiting same state and scenario after reject, and gradient not being available
    state, correction = hmc.hints(state, args.levels, always_accept = aa) # e.g. dbg = (t==0)
    # diagnostic histogram
    # show progress
    prev_then = then
    then = time.time()
    t+=1
    if True:
        with torch.no_grad():
            loss = MH.minibatch_loss(state, (t-1) % NUM_SCENARIOS)
        correct = 0
        total = 0
        for images, labels in test_loader:
            images = Variable(images.view(-1, 28*28))
            outputs = state(images.to(device))
            # TO DO convert these outputs to probs, to get more accurate Accuracy metric
            # and to support historical averaging (e.g. q(n) = p(n) * 2/(n+1) + q(n-1) * (n-1)/(n+1))
            _, predicted = torch.max(outputs.cpu().data, 1)
            total += labels.size(0)
            # for gpu, bring the predicted and labels back to cpu fro python operations to work
            correct+= (predicted == labels).sum()
        accuracy = 100 * float(correct)/total
        prev_now = now
        now = time.time()
        print("Iteration: {}. Train time:{}. Test time:{}. Noisy loss: {}. Accuracy: {}."\
              .format(t, round(then - prev_now, 2), round(now - then, 2), round(loss.item(),2), accuracy), hmc.acceptances, hmc.rejections)
    else:
        now = time.time()
        print("Iteration: {}. Time taken:{}."\
              .format(t, round(then - prev_then, 2), hmc.acceptances, hmc.rejections))

# print time per evaluation
hmc.acceptances.sum()


1716904181.2081645
0
Iteration: 1. Train time:102.31. Test time:1.67. Noisy loss: 85.35. Accuracy: 83.44. [389 300 100  50  10   5   1] [211   0   0   0   0   0   0]
1
Iteration: 2. Train time:104.16. Test time:1.66. Noisy loss: 67.72. Accuracy: 86.5. [885 599 199 100  20  10   2] [315   1   1   0   0   0   0]
2
Iteration: 3. Train time:104.99. Test time:1.67. Noisy loss: 54.3. Accuracy: 87.55. [1407  892  299  148   30   15    3] [393   8   1   2   0   0   0]
3
Iteration: 4. Train time:105.23. Test time:1.67. Noisy loss: 31.78. Accuracy: 88.21. [1954 1185  399  198   40   20    4] [446  15   1   2   0   0   0]
4
Iteration: 5. Train time:104.29. Test time:1.66. Noisy loss: 44.66. Accuracy: 88.78. [2499 1478  498  246   50   25    5] [501  22   2   4   0   0   0]
5
Iteration: 6. Train time:103.71. Test time:1.66. Noisy loss: 52.6. Accuracy: 89.08. [3035 1769  594  295   60   30    6] [565  31   6   5   0   0   0]
6
Iteration: 7. Train time:104.05. Test time:1.67. Noisy loss: 58.94. Accu

11785

In [11]:
print(g.total_counter, g.counter, "miss% = " + str(int((100.0 * g.counter)/ g.total_counter))) # check cache ratio
#(len(set(g.calls)), len(g.calls)) # check potential gain from cache

16800 12394 miss% = 73


In [12]:
g.evaluate.cache_info()

CacheInfo(hits=2136, misses=6264, maxsize=1000000, currsize=6264)

In [13]:
# looks to be better than MCMC [but for this dataset, no compelling case for MCMC methods]
# HOW MUCH NOISE AT PRIMITIVE LEVEL? - zero case for SGD
# aim for bigger moves at primitive level (or HMC chain?)
# GPU - DONE
# Bayesian accuracy measure thru decaying average (triangle distrib)
